# Agents in LlamaIndex

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

## Let's install the dependencies

We will install the dependencies for this unit.

In [1]:
!pip install llama-index llama-index-vector-stores-chroma llama-index-llms-huggingface-api llama-index-embeddings-huggingface -U -q

You should consider upgrading via the '/Users/ron/Documents/github/myenv/bin/python3 -m pip install --upgrade pip' command.


And, let's log in to Hugging Face to use serverless Inference APIs.

In [12]:
from huggingface_hub import login

login()

## Initialising agents

Let's start by initialising an agent. We will use the basic `AgentWorkflow` class to create an agent.

In [13]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent.workflow import AgentWorkflow, ToolCallResult, AgentStream


def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b


llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct", provider="auto")
#llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct", provider="auto")

agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[subtract, multiply, divide, add],
    llm=llm,
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools.",
)

Then, we can run the agent and get the response and reasoning behind the tool calls.

In [14]:
handler = agent.run("What is (2 + 2) * 3?")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 2, "b": 2}

Called tool:  add {'a': 2, 'b': 2} => 4
Thought: Now I need to multiply the result by 3.
Action: multiply
Action Input: {'a': 4, 'b': 3}
Called tool:  multiply {'a': 4, 'b': 3} => 12
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: (2 + 2) * 3 = 12

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='(2 + 2) * 3 = 12')]), structured_response=None, current_agent_name='Agent', raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role=None, content='', tool_call_id=None, tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1753563735953, id='BMZKPq', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint=None, usage=None, object='chat.completion.chunk'), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 2, 'b': 2}, tool_id='d42f0d5d-b353-4b92-acee-d0aeadc470c2', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='4')], tool_name='add', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False), ToolCallResult(tool_name='multiply', tool_kwargs={'a': 4, 'b': 3}, tool_id='1d25677a-adf5-4f40-b8e5-4a7

In a similar fashion, we can pass state and context to the agent.


In [5]:
from llama_index.core.workflow import Context
# stateless
response = await agent.run("What is 2 times 2?")

ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Your name is Bob.')]), structured_response=None, current_agent_name='Agent', raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role=None, content='', tool_call_id=None, tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1753039245364, id='fqfziv', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint=None, usage=None, object='chat.completion.chunk'), tool_calls=[], retry_messages=[])

In [15]:
# stateless
response = await agent.run("What is 2 times 2?")

# remembering state
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)
print(response)

Your name is Bob.


## Creating RAG Agents with QueryEngineTools

Let's now re-use the `QueryEngine` we defined in the [previous unit on tools](/tools.ipynb) and convert it into a `QueryEngineTool`. We will pass it to the `AgentWorkflow` class to create a RAG agent.

In [8]:
import chromadb

from llama_index.core import VectorStoreIndex
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.vector_stores.chroma import ChromaVectorStore

# Create a vector store
db = chromadb.PersistentClient(path="./alfred_chroma_db")
chroma_collection = db.get_or_create_collection("alfred")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Create a query engine
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
llm = HuggingFaceInferenceAPI(model_name="Qwen/Qwen2.5-Coder-32B-Instruct", provider="auto")
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=embed_model
)
query_engine = index.as_query_engine(llm=llm)
query_engine_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="personas",
    description="descriptions for various types of personas",
    return_direct=False,
)

# Create a RAG agent
query_engine_agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[query_engine_tool],
    llm=llm,
    system_prompt="You are a helpful assistant that has access to a database containing persona descriptions. ",
)

And, we can once more get the response and reasoning behind the tool calls.

In [9]:
handler = query_engine_agent.run(
    "Search the database for 'science fiction' and return some persona descriptions."
)
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: personas
Action Input: {"input": "science fiction"}
Called tool:  personas {'input': 'science fiction'} => science fiction
Thought: The observation provided does not seem to contain specific persona descriptions related to science fiction. I need to refine my approach to get more detailed and relevant information about personas in the context of science fiction.
Action: personas
Action Input: {'input': 'descriptions of personas in science fiction'}
Called tool:  personas {'input': 'descriptions of personas in science fiction'} => Descriptions of personas in science fiction often delve into detailed narratives that explore various aspects of human nature, societal structures, and technological advancements. These personas can range from complex, morally ambiguous characters to idealized heroes or villains, each contributing to the thematic depth of the story. They might in

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Descriptions of personas in science fiction often delve into detailed narratives that explore various aspects of human nature, societal structures, and technological advancements. These personas can range from complex, morally ambiguous characters to idealized heroes or villains, each contributing to the thematic depth of the story. They might inhabit futuristic worlds, alternate realities, or distant planets, facing challenges that test their resilience, ethics, and adaptability. Through these characters, science fiction often examines themes such as the impact of technology on society, the consequences of human actions, and the quest for knowledge and survival.')]), structured_response=None, current_agent_name='Agent', raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role=None, conten

## Creating multi-agent systems

We can also create multi-agent systems by passing multiple agents to the `AgentWorkflow` class.

In [10]:
from llama_index.core.agent.workflow import (
    AgentWorkflow,
    ReActAgent,
)


# Define some tools
def add(a: int, b: int) -> int:
    """Add two numbers."""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers."""
    return a - b


# Create agent configs
# NOTE: we can use FunctionAgent or ReActAgent here.
# FunctionAgent works for LLMs with a function calling API.
# ReActAgent works for any LLM.
calculator_agent = ReActAgent(
    name="calculator",
    description="Performs basic arithmetic operations",
    system_prompt="You are a calculator assistant. Use your tools for any math operation.",
    tools=[add, subtract],
    llm=llm,
)

query_agent = ReActAgent(
    name="info_lookup",
    description="Looks up information about XYZ",
    system_prompt="Use your tool to query a RAG system to answer information about XYZ",
    tools=[query_engine_tool],
    llm=llm,
)

# Create and run the workflow
agent = AgentWorkflow(agents=[calculator_agent, query_agent], root_agent="calculator")

# Run the system
handler = agent.run(user_msg="Can you add 5 and 3?")

In [11]:
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 5, "b": 3}
Called tool:  add {'a': 5, 'b': 3} => 8
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 8

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='8')]), structured_response=None, current_agent_name='calculator', raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role=None, content='', tool_call_id=None, tool_calls=None), index=0, finish_reason=None, logprobs=None)], created=1753039972443, id='NiiC8E', model='Qwen/Qwen2.5-Coder-32B-Instruct', system_fingerprint=None, usage=None, object='chat.completion.chunk'), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 5, 'b': 3}, tool_id='0918e328-cee8-4434-b49c-65d95bf90761', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='8')], tool_name='add', raw_input={'args': (), 'kwargs': {'a': 5, 'b': 3}}, raw_output=8, is_error=False), return_direct=False)], retry_messages=[])